# Equinor news

Denne notebooken er veldig lang, og overkomplisert, kun fordi det tok utrolig lang tid å gå gjennom nettsiden til Equinor og scrolle gjennom alle sidene - vi fant ingen effektiv måte å gjøre dette på. Så det er relativt likt de andre webscraping-filene, bare gjort om og om igjen. 

## Imports

In [41]:
# Uninstall the problematic packages
%pip uninstall torch sentence-transformers -y

# Reinstall the packages
%pip install torch sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: torch 2.3.1
Uninstalling torch-2.3.1:
  Successfully uninstalled torch-2.3.1
Found existing installation: sentence-transformers 3.0.1
Uninstalling sentence-transformers-3.0.1:
  Successfully uninstalled sentence-transformers-3.0.1
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Loading egg at /opt/anaconda3/envs/embed/lib/python3.12/site-packages/sacremoses-0.0.43-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Using cached torch-2.3.1-cp312-none-macosx_11_0_arm64.whl.metadata (26 kB)
  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
Using cached torch-2.3.1-cp312-none-macosx_11_0_arm64.whl (61.0 MB)
Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time
import pickle
import requests
from sentence_transformers import SentenceTransformer

/opt/anaconda3/envs/embed/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Finner url-er

In [43]:
## 2024-2019 (can only go 50 pages at a time)

# Initialize WebDriver (e.g., for Chrome)
driver = webdriver.Chrome()

def read_urls(url):
    driver.get(url)
    
    article_links_by_year = {}  # Use a normal dictionary
    
    while True:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        # Extract date headers and their respective links
        date_spans = soup.find_all('span', class_='sc-aac650bd-0 dakTuR')
        news_items = soup.find_all('li', class_='sc-db713ca2-0 ezdnkb sc-41ce2e98-1 dYECJz ais-Hits-item')

        for date_span, news_item in zip(date_spans, news_items):
            date_text = date_span.get_text().strip()
            print(f"Found date span: {date_text}")  # Debugging line to check the date text

            # Check if date_text contains a valid year
            if len(date_text.split()) > 1:
                year = date_text.split()[-1]  # Extract the year from the date text
                print(f"Extracted year: {year}")  # Debugging line to check the extracted year

                # Ensure the year key exists in the dictionary
                if year not in article_links_by_year:
                    article_links_by_year[year] = []

                # Extracting the 'a' tags within the news item
                for a_tag in news_item.find_all('a', href=True):
                    href = a_tag['href']
                    full_url = f"https://www.equinor.com{href}" if href.startswith('/') else href
                    print(f"Found URL link: {full_url}")  # Debugging line to check found links

                    if full_url not in article_links_by_year[year]:
                        article_links_by_year[year].append(full_url)
                        print(f"Added URL link: {full_url} under year {year}")  # Debugging line to check added links
            else:
                print(f"Invalid date format: {date_text}")
        
        # Check if the next page button is available and click it
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Next"]'))
            )
            # Check if the next button is disabled
            if 'disabled' in next_button.get_attribute('class'):
                print("Next button is disabled. No more pages to load.")
                break
            # Scroll the next button into view
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            # Use ActionChains to click the button
            ActionChains(driver).move_to_element(next_button).click(next_button).perform()
            time.sleep(2)  # Wait for content to load
        except Exception as e:
            print(f"No more 'Next' button found or error occurred: {e}")
            break  # No more "Next" button found or an error occurred
    
    return article_links_by_year

equinor_news_url1 = 'https://www.equinor.com/news?years=2024&years=2023&years=2022&years=2021&years=2020&years=2019'
article_links_by_year = read_urls(equinor_news_url1)

# Close the WebDriver
driver.quit()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found date span: 24 July 2024
Extracted year: 2024
Found URL link: https://www.equinor.com/news/equinor-second-quarter-2024-results
Added URL link: https://www.equinor.com/news/equinor-second-quarter-2024-results under year 2024
Found date span: 24 July 2024
Extracted year: 2024
Found URL link: https://www.equinor.com/news/20240724-third-tranche-2024-share-buy-back-programme
Added URL link: https://www.equinor.com/news/20240724-third-tranche-2024-share-buy-back-programme under year 2024
Found date span: 24 July 2024
Extracted year: 2024
Found URL link: https://www.equinor.com/news/20240724-key-information-cash-dividend-2q2024
Added URL link: https://www.equinor.com/news/20240724-key-information-cash-dividend-2q2024 under year 2024
Found date span: 22 July 2024
Extracted year: 2024
Found URL link: https://www.equinor.com/news/20240722-valaris-ds-17-raia-project-brazil
Added URL link: https://www.equinor.com/news/20240722-valaris-ds-17-raia-project-brazil under year 2024
Found date span:

In [44]:
article_links_by_year["2019"][-1]

'https://www.equinor.com/en/news/archive/2019-01-07-digital-operations-support-centres'

In [45]:
article_links_by_year.keys()

dict_keys(['2024', '2023', '2022', '2021', '2020', '2019'])

In [46]:
## 2018-2013 (can only go 50 pages at a time)

# Initialize WebDriver (e.g., for Chrome)
driver = webdriver.Chrome()

def read_urls(url, article_links_by_year):
    driver.get(url)
    
    while True:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        # Extract date headers and their respective links
        date_spans = soup.find_all('span', class_='sc-aac650bd-0 dakTuR')
        news_items = soup.find_all('li', class_='sc-db713ca2-0 ezdnkb sc-41ce2e98-1 dYECJz ais-Hits-item')

        for date_span, news_item in zip(date_spans, news_items):
            date_text = date_span.get_text().strip()
            print(f"Found date span: {date_text}")  # Debugging line to check the date text

            # Check if date_text contains a valid year
            if len(date_text.split()) > 1:
                year = date_text.split()[-1]  # Extract the year from the date text
                print(f"Extracted year: {year}")  # Debugging line to check the extracted year

                # Ensure the year key exists in the dictionary
                if year not in article_links_by_year:
                    article_links_by_year[year] = []

                # Extracting the 'a' tags within the news item
                for a_tag in news_item.find_all('a', href=True):
                    href = a_tag['href']
                    full_url = f"https://www.equinor.com{href}" if href.startswith('/') else href
                    print(f"Found URL link: {full_url}")  # Debugging line to check found links

                    if full_url not in article_links_by_year[year]:
                        article_links_by_year[year].append(full_url)
                        print(f"Added URL link: {full_url} under year {year}")  # Debugging line to check added links
            else:
                print(f"Invalid date format: {date_text}")
        
        # Check if the next page button is available and click it
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Next"]'))
            )
            # Check if the next button is disabled
            if 'disabled' in next_button.get_attribute('class'):
                print("Next button is disabled. No more pages to load.")
                break
            # Scroll the next button into view
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            # Use ActionChains to click the button
            ActionChains(driver).move_to_element(next_button).click(next_button).perform()
            time.sleep(2)  # Wait for content to load
        except Exception as e:
            print(f"No more 'Next' button found or error occurred: {e}")
            break  # No more "Next" button found or an error occurred
    
    return article_links_by_year

equinor_news_url2 = 'https://www.equinor.com/news?years=2018&years=2017&years=2016&years=2015&years=2014&years=2013'
article_links_by_year = read_urls(equinor_news_url2,article_links_by_year)

# Close the WebDriver
driver.quit()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found date span: 28 December 2018
Extracted year: 2018
Found URL link: https://www.equinor.com/en/news/archive/sale-tommeliten-king-lear
Added URL link: https://www.equinor.com/en/news/archive/sale-tommeliten-king-lear under year 2018
Found date span: 20 December 2018
Extracted year: 2018
Found URL link: https://www.equinor.com/en/news/archive/2018-december-offshore-wind-poland
Added URL link: https://www.equinor.com/en/news/archive/2018-december-offshore-wind-poland under year 2018
Found date span: 19 December 2018
Extracted year: 2018
Found URL link: https://www.equinor.com/en/news/archive/2018-12-19-johan-sverdrup
Added URL link: https://www.equinor.com/en/news/archive/2018-12-19-johan-sverdrup under year 2018
Found date span: 17 December 2018
Extracted year: 2018
Found URL link: https://www.equinor.com/en/news/archive/2018-12-17-aasta-hansteen
Added URL link: https://www.equinor.com/en/news/archive/2018-12-17-aasta-hansteen under year 2018
Found date span: 14 December 2018
Extracte

In [47]:
article_links_by_year.keys()

dict_keys(['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013'])

In [48]:
## 2012-2008 (can only go 50 pages at a time)

# Initialize WebDriver (e.g., for Chrome)
driver = webdriver.Chrome()

def read_urls(url, article_links_by_year):
    driver.get(url)
    
    while True:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        # Extract date headers and their respective links
        date_spans = soup.find_all('span', class_='sc-aac650bd-0 dakTuR')
        news_items = soup.find_all('li', class_='sc-db713ca2-0 ezdnkb sc-41ce2e98-1 dYECJz ais-Hits-item')

        for date_span, news_item in zip(date_spans, news_items):
            date_text = date_span.get_text().strip()
            print(f"Found date span: {date_text}")  # Debugging line to check the date text

            # Check if date_text contains a valid year
            if len(date_text.split()) > 1:
                year = date_text.split()[-1]  # Extract the year from the date text
                print(f"Extracted year: {year}")  # Debugging line to check the extracted year

                # Ensure the year key exists in the dictionary
                if year not in article_links_by_year:
                    article_links_by_year[year] = []

                # Extracting the 'a' tags within the news item
                for a_tag in news_item.find_all('a', href=True):
                    href = a_tag['href']
                    full_url = f"https://www.equinor.com{href}" if href.startswith('/') else href
                    print(f"Found URL link: {full_url}")  # Debugging line to check found links

                    if full_url not in article_links_by_year[year]:
                        article_links_by_year[year].append(full_url)
                        print(f"Added URL link: {full_url} under year {year}")  # Debugging line to check added links
            else:
                print(f"Invalid date format: {date_text}")
        
        # Check if the next page button is available and click it
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Next"]'))
            )
            # Check if the next button is disabled
            if 'disabled' in next_button.get_attribute('class'):
                print("Next button is disabled. No more pages to load.")
                break
            # Scroll the next button into view
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            # Use ActionChains to click the button
            ActionChains(driver).move_to_element(next_button).click(next_button).perform()
            time.sleep(2)  # Wait for content to load
        except Exception as e:
            print(f"No more 'Next' button found or error occurred: {e}")
            break  # No more "Next" button found or an error occurred
    
    return article_links_by_year

equinor_news_url3 = 'https://www.equinor.com/news?years=2012&years=2011&years=2010&years=2009&years=2008'
article_links_by_year = read_urls(equinor_news_url3,article_links_by_year)

# Close the WebDriver
driver.quit()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found date span: 27 December 2012
Extracted year: 2012
Found URL link: https://www.equinor.com/en/news/archive/2012/12/27/27DecVale
Added URL link: https://www.equinor.com/en/news/archive/2012/12/27/27DecVale under year 2012
Found date span: 21 December 2012
Extracted year: 2012
Found URL link: https://www.equinor.com/en/news/archive/2012/12/21/21DecMariner
Added URL link: https://www.equinor.com/en/news/archive/2012/12/21/21DecMariner under year 2012
Found date span: 21 December 2012
Extracted year: 2012
Found URL link: https://www.equinor.com/en/news/archive/2012/12/21/20DecLavani2
Added URL link: https://www.equinor.com/en/news/archive/2012/12/21/20DecLavani2 under year 2012
Found date span: 20 December 2012
Extracted year: 2012
Found URL link: https://www.equinor.com/en/news/archive/2012/12/20/20Dec2012NotifiableTrading
Added URL link: https://www.equinor.com/en/news/archive/2012/12/20/20Dec2012NotifiableTrading under year 2012
Found date span: 20 December 2012
Extracted year: 2012

In [49]:
article_links_by_year.keys()

dict_keys(['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008'])

In [50]:
## 2007-2006 (can only go 50 pages at a time)

# Initialize WebDriver (e.g., for Chrome)
driver = webdriver.Chrome()

def read_urls(url, article_links_by_year):
    driver.get(url)
    
    while True:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        # Extract date headers and their respective links
        date_spans = soup.find_all('span', class_='sc-aac650bd-0 dakTuR')
        news_items = soup.find_all('li', class_='sc-db713ca2-0 ezdnkb sc-41ce2e98-1 dYECJz ais-Hits-item')

        for date_span, news_item in zip(date_spans, news_items):
            date_text = date_span.get_text().strip()
            print(f"Found date span: {date_text}")  # Debugging line to check the date text

            # Check if date_text contains a valid year
            if len(date_text.split()) > 1:
                year = date_text.split()[-1]  # Extract the year from the date text
                print(f"Extracted year: {year}")  # Debugging line to check the extracted year

                # Ensure the year key exists in the dictionary
                if year not in article_links_by_year:
                    article_links_by_year[year] = []

                # Extracting the 'a' tags within the news item
                for a_tag in news_item.find_all('a', href=True):
                    href = a_tag['href']
                    full_url = f"https://www.equinor.com{href}" if href.startswith('/') else href
                    print(f"Found URL link: {full_url}")  # Debugging line to check found links

                    if full_url not in article_links_by_year[year]:
                        article_links_by_year[year].append(full_url)
                        print(f"Added URL link: {full_url} under year {year}")  # Debugging line to check added links
            else:
                print(f"Invalid date format: {date_text}")
        
        # Check if the next page button is available and click it
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Next"]'))
            )
            # Check if the next button is disabled
            if 'disabled' in next_button.get_attribute('class'):
                print("Next button is disabled. No more pages to load.")
                break
            # Scroll the next button into view
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            # Use ActionChains to click the button
            ActionChains(driver).move_to_element(next_button).click(next_button).perform()
            time.sleep(2)  # Wait for content to load
        except Exception as e:
            print(f"No more 'Next' button found or error occurred: {e}")
            break  # No more "Next" button found or an error occurred
    
    return article_links_by_year

equinor_news_url4 = 'https://www.equinor.com/news?years=2007&years=2006'
article_links_by_year = read_urls(equinor_news_url4,article_links_by_year)

# Close the WebDriver
driver.quit()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found date span: 29 December 2007
Extracted year: 2007
Found URL link: https://www.equinor.com/en/news/archive/2007/12/29/GulfMariner
Added URL link: https://www.equinor.com/en/news/archive/2007/12/29/GulfMariner under year 2007
Found date span: 28 December 2007
Extracted year: 2007
Found URL link: https://www.equinor.com/en/news/archive/2007/12/28/PazflorDevelopment
Added URL link: https://www.equinor.com/en/news/archive/2007/12/28/PazflorDevelopment under year 2007
Found date span: 21 December 2007
Extracted year: 2007
Found URL link: https://www.equinor.com/en/news/archive/2007/12/21/PeregrinoOperatorship
Added URL link: https://www.equinor.com/en/news/archive/2007/12/21/PeregrinoOperatorship under year 2007
Found date span: 20 December 2007
Extracted year: 2007
Found URL link: https://www.equinor.com/en/news/archive/2007/12/20/NameChange2009
Added URL link: https://www.equinor.com/en/news/archive/2007/12/20/NameChange2009 under year 2007
Found date span: 18 December 2007
Extracted 

In [51]:
article_links_by_year.keys()

dict_keys(['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006'])

In [52]:
## 2005-2004 (can only go 50 pages at a time)

# Initialize WebDriver (e.g., for Chrome)
driver = webdriver.Chrome()

def read_urls(url, article_links_by_year):
    driver.get(url)
    
    while True:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        # Extract date headers and their respective links
        date_spans = soup.find_all('span', class_='sc-aac650bd-0 dakTuR')
        news_items = soup.find_all('li', class_='sc-db713ca2-0 ezdnkb sc-41ce2e98-1 dYECJz ais-Hits-item')

        for date_span, news_item in zip(date_spans, news_items):
            date_text = date_span.get_text().strip()
            print(f"Found date span: {date_text}")  # Debugging line to check the date text

            # Check if date_text contains a valid year
            if len(date_text.split()) > 1:
                year = date_text.split()[-1]  # Extract the year from the date text
                print(f"Extracted year: {year}")  # Debugging line to check the extracted year

                # Ensure the year key exists in the dictionary
                if year not in article_links_by_year:
                    article_links_by_year[year] = []

                # Extracting the 'a' tags within the news item
                for a_tag in news_item.find_all('a', href=True):
                    href = a_tag['href']
                    full_url = f"https://www.equinor.com{href}" if href.startswith('/') else href
                    print(f"Found URL link: {full_url}")  # Debugging line to check found links

                    if full_url not in article_links_by_year[year]:
                        article_links_by_year[year].append(full_url)
                        print(f"Added URL link: {full_url} under year {year}")  # Debugging line to check added links
            else:
                print(f"Invalid date format: {date_text}")
        
        # Check if the next page button is available and click it
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Next"]'))
            )
            # Check if the next button is disabled
            if 'disabled' in next_button.get_attribute('class'):
                print("Next button is disabled. No more pages to load.")
                break
            # Scroll the next button into view
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            # Use ActionChains to click the button
            ActionChains(driver).move_to_element(next_button).click(next_button).perform()
            time.sleep(2)  # Wait for content to load
        except Exception as e:
            print(f"No more 'Next' button found or error occurred: {e}")
            break  # No more "Next" button found or an error occurred
    
    return article_links_by_year

equinor_news_url5 = 'https://www.equinor.com/news?years=2005&years=2004'
article_links_by_year = read_urls(equinor_news_url5,article_links_by_year)

# Close the WebDriver
driver.quit()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found date span: 29 December 2005
Extracted year: 2005
Found URL link: https://www.equinor.com/en/news/archive/2005/12/29/UsingFewerHarmfulChemicals
Added URL link: https://www.equinor.com/en/news/archive/2005/12/29/UsingFewerHarmfulChemicals under year 2005
Found date span: 28 December 2005
Extracted year: 2005
Found URL link: https://www.equinor.com/en/news/archive/2005/12/28/AirAccordWithSASBraathensWidereAndStarAlliance
Added URL link: https://www.equinor.com/en/news/archive/2005/12/28/AirAccordWithSASBraathensWidereAndStarAlliance under year 2005
Found date span: 27 December 2005
Extracted year: 2005
Found URL link: https://www.equinor.com/en/news/archive/2005/12/27/ActiveSeismicSeason
Added URL link: https://www.equinor.com/en/news/archive/2005/12/27/ActiveSeismicSeason under year 2005
Found date span: 23 December 2005
Extracted year: 2005
Found URL link: https://www.equinor.com/en/news/archive/2005/12/23/HydroInvestsInElectricityAndGasTradingSystem
Added URL link: https://www.eq

In [53]:
article_links_by_year.keys()

dict_keys(['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004'])

In [54]:
## 2003-2002 (can only go 50 pages at a time)

# Initialize WebDriver (e.g., for Chrome)
driver = webdriver.Chrome()

def read_urls(url, article_links_by_year):
    driver.get(url)
    
    while True:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        # Extract date headers and their respective links
        date_spans = soup.find_all('span', class_='sc-aac650bd-0 dakTuR')
        news_items = soup.find_all('li', class_='sc-db713ca2-0 ezdnkb sc-41ce2e98-1 dYECJz ais-Hits-item')

        for date_span, news_item in zip(date_spans, news_items):
            date_text = date_span.get_text().strip()
            print(f"Found date span: {date_text}")  # Debugging line to check the date text

            # Check if date_text contains a valid year
            if len(date_text.split()) > 1:
                year = date_text.split()[-1]  # Extract the year from the date text
                print(f"Extracted year: {year}")  # Debugging line to check the extracted year

                # Ensure the year key exists in the dictionary
                if year not in article_links_by_year:
                    article_links_by_year[year] = []

                # Extracting the 'a' tags within the news item
                for a_tag in news_item.find_all('a', href=True):
                    href = a_tag['href']
                    full_url = f"https://www.equinor.com{href}" if href.startswith('/') else href
                    print(f"Found URL link: {full_url}")  # Debugging line to check found links

                    if full_url not in article_links_by_year[year]:
                        article_links_by_year[year].append(full_url)
                        print(f"Added URL link: {full_url} under year {year}")  # Debugging line to check added links
            else:
                print(f"Invalid date format: {date_text}")
        
        # Check if the next page button is available and click it
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Next"]'))
            )
            # Check if the next button is disabled
            if 'disabled' in next_button.get_attribute('class'):
                print("Next button is disabled. No more pages to load.")
                break
            # Scroll the next button into view
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            # Use ActionChains to click the button
            ActionChains(driver).move_to_element(next_button).click(next_button).perform()
            time.sleep(2)  # Wait for content to load
        except Exception as e:
            print(f"No more 'Next' button found or error occurred: {e}")
            break  # No more "Next" button found or an error occurred
    
    return article_links_by_year

equinor_news_url5 = 'https://www.equinor.com/news?years=2003&years=2002'
article_links_by_year = read_urls(equinor_news_url5,article_links_by_year)

# Close the WebDriver
driver.quit()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found date span: 30 December 2003
Extracted year: 2003
Found URL link: https://www.equinor.com/en/news/archive/2003/12/30/ThroughtubingWellsCutCosts
Added URL link: https://www.equinor.com/en/news/archive/2003/12/30/ThroughtubingWellsCutCosts under year 2003
Found date span: 23 December 2003
Extracted year: 2003
Found URL link: https://www.equinor.com/en/news/archive/2003/12/23/AlgeriaAcquisitionApproved
Added URL link: https://www.equinor.com/en/news/archive/2003/12/23/AlgeriaAcquisitionApproved under year 2003
Found date span: 23 December 2003
Extracted year: 2003
Found URL link: https://www.equinor.com/en/news/archive/2003/12/23/PronovaBiocareToBeSold
Added URL link: https://www.equinor.com/en/news/archive/2003/12/23/PronovaBiocareToBeSold under year 2003
Found date span: 19 December 2003
Extracted year: 2003
Found URL link: https://www.equinor.com/en/news/archive/2003/12/19/OrmenLangeRockInstallationContractAwarded
Added URL link: https://www.equinor.com/en/news/archive/2003/12/19/

In [55]:
article_links_by_year.keys()

dict_keys(['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003'])

In [56]:
## 2001-1999 (can only go 50 pages at a time)

# Initialize WebDriver (e.g., for Chrome)
driver = webdriver.Chrome()

def read_urls(url, article_links_by_year):
    driver.get(url)
    
    while True:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        # Extract date headers and their respective links
        date_spans = soup.find_all('span', class_='sc-aac650bd-0 dakTuR')
        news_items = soup.find_all('li', class_='sc-db713ca2-0 ezdnkb sc-41ce2e98-1 dYECJz ais-Hits-item')

        for date_span, news_item in zip(date_spans, news_items):
            date_text = date_span.get_text().strip()
            print(f"Found date span: {date_text}")  # Debugging line to check the date text

            # Check if date_text contains a valid year
            if len(date_text.split()) > 1:
                year = date_text.split()[-1]  # Extract the year from the date text
                print(f"Extracted year: {year}")  # Debugging line to check the extracted year

                # Ensure the year key exists in the dictionary
                if year not in article_links_by_year:
                    article_links_by_year[year] = []

                # Extracting the 'a' tags within the news item
                for a_tag in news_item.find_all('a', href=True):
                    href = a_tag['href']
                    full_url = f"https://www.equinor.com{href}" if href.startswith('/') else href
                    print(f"Found URL link: {full_url}")  # Debugging line to check found links

                    if full_url not in article_links_by_year[year]:
                        article_links_by_year[year].append(full_url)
                        print(f"Added URL link: {full_url} under year {year}")  # Debugging line to check added links
            else:
                print(f"Invalid date format: {date_text}")
        
        # Check if the next page button is available and click it
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Next"]'))
            )
            # Check if the next button is disabled
            if 'disabled' in next_button.get_attribute('class'):
                print("Next button is disabled. No more pages to load.")
                break
            # Scroll the next button into view
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            # Use ActionChains to click the button
            ActionChains(driver).move_to_element(next_button).click(next_button).perform()
            time.sleep(2)  # Wait for content to load
        except Exception as e:
            print(f"No more 'Next' button found or error occurred: {e}")
            break  # No more "Next" button found or an error occurred
    
    return article_links_by_year

equinor_news_url6 = 'https://www.equinor.com/news?years=2001&years=2000&years=1999'
article_links_by_year = read_urls(equinor_news_url6,article_links_by_year)

# Close the WebDriver
driver.quit()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found date span: 21 December 2001
Extracted year: 2001
Found URL link: https://www.equinor.com/en/news/archive/2001/12/21/TakingOverStenaDon
Added URL link: https://www.equinor.com/en/news/archive/2001/12/21/TakingOverStenaDon under year 2001
Found date span: 21 December 2001
Extracted year: 2001
Found URL link: https://www.equinor.com/en/news/archive/2001/12/21/GullfaksHits15thAnniversary
Added URL link: https://www.equinor.com/en/news/archive/2001/12/21/GullfaksHits15thAnniversary under year 2001
Found date span: 21 December 2001
Extracted year: 2001
Found URL link: https://www.equinor.com/en/news/archive/2001/12/21/UnitisationApproved
Added URL link: https://www.equinor.com/en/news/archive/2001/12/21/UnitisationApproved under year 2001
Found date span: 20 December 2001
Extracted year: 2001
Found URL link: https://www.equinor.com/en/news/archive/2001/12/20/NewEnvironmentVicePresident
Added URL link: https://www.equinor.com/en/news/archive/2001/12/20/NewEnvironmentVicePresident under 

In [57]:
article_links_by_year.keys()

dict_keys(['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003', '2001', '2000', '1999'])

In [58]:
## 1998 (can only go 50 pages at a time)

# Initialize WebDriver (e.g., for Chrome)
driver = webdriver.Chrome()

def read_urls(url, article_links_by_year):
    driver.get(url)
    
    while True:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        # Extract date headers and their respective links
        date_spans = soup.find_all('span', class_='sc-aac650bd-0 dakTuR')
        news_items = soup.find_all('li', class_='sc-db713ca2-0 ezdnkb sc-41ce2e98-1 dYECJz ais-Hits-item')

        for date_span, news_item in zip(date_spans, news_items):
            date_text = date_span.get_text().strip()
            print(f"Found date span: {date_text}")  # Debugging line to check the date text

            # Check if date_text contains a valid year
            if len(date_text.split()) > 1:
                year = date_text.split()[-1]  # Extract the year from the date text
                print(f"Extracted year: {year}")  # Debugging line to check the extracted year

                # Ensure the year key exists in the dictionary
                if year not in article_links_by_year:
                    article_links_by_year[year] = []

                # Extracting the 'a' tags within the news item
                for a_tag in news_item.find_all('a', href=True):
                    href = a_tag['href']
                    full_url = f"https://www.equinor.com{href}" if href.startswith('/') else href
                    print(f"Found URL link: {full_url}")  # Debugging line to check found links

                    if full_url not in article_links_by_year[year]:
                        article_links_by_year[year].append(full_url)
                        print(f"Added URL link: {full_url} under year {year}")  # Debugging line to check added links
            else:
                print(f"Invalid date format: {date_text}")
        
        # Check if the next page button is available and click it
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Next"]'))
            )
            # Check if the next button is disabled
            if 'disabled' in next_button.get_attribute('class'):
                print("Next button is disabled. No more pages to load.")
                break
            # Scroll the next button into view
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            # Use ActionChains to click the button
            ActionChains(driver).move_to_element(next_button).click(next_button).perform()
            time.sleep(2)  # Wait for content to load
        except Exception as e:
            print(f"No more 'Next' button found or error occurred: {e}")
            break  # No more "Next" button found or an error occurred
    
    return article_links_by_year

equinor_news_url7 = 'https://www.equinor.com/news?years=1998'
article_links_by_year = read_urls(equinor_news_url7,article_links_by_year)

# Close the WebDriver
driver.quit()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found date span: 29 December 1998
Extracted year: 1998
Found URL link: https://www.equinor.com/en/news/archive/1998/12/29/DrillingNearNorne
Added URL link: https://www.equinor.com/en/news/archive/1998/12/29/DrillingNearNorne under year 1998
Found date span: 24 December 1998
Extracted year: 1998
Found URL link: https://www.equinor.com/en/news/archive/1998/12/24/RigReaction
Added URL link: https://www.equinor.com/en/news/archive/1998/12/24/RigReaction under year 1998
Found date span: 23 December 1998
Extracted year: 1998
Found URL link: https://www.equinor.com/en/news/archive/1998/12/23/VargOnStream
Added URL link: https://www.equinor.com/en/news/archive/1998/12/23/VargOnStream under year 1998
Found date span: 23 December 1998
Extracted year: 1998
Found URL link: https://www.equinor.com/en/news/archive/1998/12/23/SafeShip
Added URL link: https://www.equinor.com/en/news/archive/1998/12/23/SafeShip under year 1998
Found date span: 22 December 1998
Extracted year: 1998
Found URL link: https

In [ ]:
# 2002

## 1998 (can only go 50 pages at a time)

# Initialize WebDriver (e.g., for Chrome)
driver = webdriver.Chrome()

def read_urls(url, article_links_by_year):
    driver.get(url)
    
    while True:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        # Extract date headers and their respective links
        date_spans = soup.find_all('span', class_='sc-aac650bd-0 dakTuR')
        news_items = soup.find_all('li', class_='sc-db713ca2-0 ezdnkb sc-41ce2e98-1 dYECJz ais-Hits-item')

        for date_span, news_item in zip(date_spans, news_items):
            date_text = date_span.get_text().strip()
            print(f"Found date span: {date_text}")  # Debugging line to check the date text

            # Check if date_text contains a valid year
            if len(date_text.split()) > 1:
                year = date_text.split()[-1]  # Extract the year from the date text
                print(f"Extracted year: {year}")  # Debugging line to check the extracted year

                # Ensure the year key exists in the dictionary
                if year not in article_links_by_year:
                    article_links_by_year[year] = []

                # Extracting the 'a' tags within the news item
                for a_tag in news_item.find_all('a', href=True):
                    href = a_tag['href']
                    full_url = f"https://www.equinor.com{href}" if href.startswith('/') else href
                    print(f"Found URL link: {full_url}")  # Debugging line to check found links

                    if full_url not in article_links_by_year[year]:
                        article_links_by_year[year].append(full_url)
                        print(f"Added URL link: {full_url} under year {year}")  # Debugging line to check added links
            else:
                print(f"Invalid date format: {date_text}")
        
        # Check if the next page button is available and click it
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Next"]'))
            )
            # Check if the next button is disabled
            if 'disabled' in next_button.get_attribute('class'):
                print("Next button is disabled. No more pages to load.")
                break
            # Scroll the next button into view
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            # Use ActionChains to click the button
            ActionChains(driver).move_to_element(next_button).click(next_button).perform()
            time.sleep(2)  # Wait for content to load
        except Exception as e:
            print(f"No more 'Next' button found or error occurred: {e}")
            break  # No more "Next" button found or an error occurred
    
    return article_links_by_year

equinor_news_url7 = 'https://www.equinor.com/news?years=1998'
article_links_by_year = read_urls(equinor_news_url7,article_links_by_year)

# Close the WebDriver
driver.quit()

In [59]:
article_links_by_year.keys()

dict_keys(['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003', '2001', '2000', '1999', '1998'])

In [60]:
for year in article_links_by_year:
    print(f"{year}: Number of articles = {len(article_links_by_year[year])}")

## OBS: det står feil på nettsidene til equinor, ikke her! :)

2024: Number of articles = 79
2023: Number of articles = 123
2022: Number of articles = 191
2021: Number of articles = 189
2020: Number of articles = 158
2019: Number of articles = 159
2018: Number of articles = 174
2017: Number of articles = 157
2016: Number of articles = 182
2015: Number of articles = 165
2014: Number of articles = 146
2013: Number of articles = 171
2012: Number of articles = 182
2011: Number of articles = 200
2010: Number of articles = 166
2009: Number of articles = 181
2008: Number of articles = 122
2007: Number of articles = 311
2006: Number of articles = 436
2005: Number of articles = 408
2004: Number of articles = 400
2003: Number of articles = 20
2001: Number of articles = 352
2000: Number of articles = 281
1999: Number of articles = 353
1998: Number of articles = 164


## Extraction

In [61]:
def extract_text_from_html(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # List of content div classes that should be processed
        content_div_classes = [
            'articleBody', 
            'text-pretty', 
            'prose',
            'text parbase section',
            'ingress text parbase section'
        ]

        content = ''
        
        # Extract text from the main title if available
        main_title = soup.find('h1')
        if main_title:
            content += main_title.get_text(separator=' ', strip=True) + ' '

        for class_name in content_div_classes:
            content_divs = soup.find_all('div', class_=class_name)
            if content_divs:
                for div in content_divs:
                    paragraphs = div.find_all(['p', 'ul', 'li'])
                    if paragraphs:
                        content += ' '.join(p.get_text(separator=' ', strip=True) for p in paragraphs) + ' '

        # Clean up the content by stripping excess whitespace
        content = ' '.join(content.split())

        return content if content.strip() else None
    except Exception as e:
        print(f"Error processing HTML URL {url}: {e}")
        return None

def extract_all_text(links_by_year):
    news_by_year = {}  # Use a normal dictionary
    for year, links in links_by_year.items():
        news_by_year[year] = []  # Initialize an empty list for each year
        for link in links:
            print(f"Processing URL: {link}")
            html_text = extract_text_from_html(link)
            if html_text:  # Only append if text was successfully extracted
                news_by_year[year].append(html_text)
    return news_by_year

In [62]:
news_by_year = extract_all_text(article_links_by_year)

Processing URL: https://www.equinor.com/news/equinor-second-quarter-2024-results


Processing URL: https://www.equinor.com/news/20240724-third-tranche-2024-share-buy-back-programme
Processing URL: https://www.equinor.com/news/20240724-key-information-cash-dividend-2q2024
Processing URL: https://www.equinor.com/news/20240722-valaris-ds-17-raia-project-brazil
Processing URL: https://www.equinor.com/news/20240717-second-quarter-2024-safety-results
Processing URL: https://www.equinor.com/news/uk/fourth-phase-of-the-world's-largest-offshore-wind-farm-dogger-bank-to-unlock-additional-renewable-power
Processing URL: https://www.equinor.com/news/uk/armed-forces-covenant
Processing URL: https://www.equinor.com/news/20240708-starts-production-from-kristin-south
Processing URL: https://www.equinor.com/news/20240708-invitation-estimates-second-quarter-financial-results
Processing URL: https://www.equinor.com/news/20240702-completed-share-capital-reduction
Processing URL: https://www.equinor.com/news/uk/dogger-bank-wind-farm-announces-new-pound25-million-commitment-to-coastal-com

In [63]:
news_by_year.keys()

dict_keys(['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003', '2001', '2000', '1999', '1998'])

In [75]:
# Initialize WebDriver (e.g., for Chrome)
driver = webdriver.Chrome()

# Variable to store links for the year 2002
links_2002 = []

# URL for Equinor news archive for the year 2002
equinor_news_url = 'https://www.equinor.com/news?years=2002'

# Start the WebDriver and load the URL
driver.get(equinor_news_url)

while True:
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # Extract date headers and their respective links
    date_spans = soup.find_all('span', class_='sc-aac650bd-0 dakTuR')
    news_items = soup.find_all('li', class_='sc-db713ca2-0 ezdnkb sc-41ce2e98-1 dYECJz ais-Hits-item')

    for date_span, news_item in zip(date_spans, news_items):
        date_text = date_span.get_text().strip()
        print(f"Found date span: {date_text}")  # Debugging line to check the date text

        # Check if date_text contains a valid year
        if len(date_text.split()) > 1:
            year = date_text.split()[-1]  # Extract the year from the date text
            print(f"Extracted year: {year}")  # Debugging line to check the extracted year

            # Only process links for the year 2002
            if year == "2002":
                # Extracting the 'a' tags within the news item
                for a_tag in news_item.find_all('a', href=True):
                    href = a_tag['href']
                    full_url = f"https://www.equinor.com{href}" if href.startswith('/') else href
                    print(f"Found URL link: {full_url}")  # Debugging line to check found links

                    if full_url not in links_2002:
                        links_2002.append(full_url)
                        print(f"Added URL link: {full_url} under year 2002")  # Debugging line to check added links
        else:
            print(f"Invalid date format: {date_text}")
    
    # Check if the next page button is available and click it
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Next"]'))
        )
        # Check if the next button is disabled
        if 'disabled' in next_button.get_attribute('class'):
            print("Next button is disabled. No more pages to load.")
            break
        # Scroll the next button into view
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        # Use ActionChains to click the button
        ActionChains(driver).move_to_element(next_button).click(next_button).perform()
        time.sleep(2)  # Wait for content to load
    except Exception as e:
        print(f"No more 'Next' button found or error occurred: {e}")
        break  # No more "Next" button found or an error occurred

# Close the WebDriver
driver.quit()

# Print the collected links for the year 2002
print(f"Links for the year 2002: {links_2002}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found date span: 31 December 2002
Extracted year: 2002
Found URL link: https://www.equinor.com/en/news/archive/2002/12/31/OperatorshipsOnTheMove
Added URL link: https://www.equinor.com/en/news/archive/2002/12/31/OperatorshipsOnTheMove under year 2002
Found date span: 23 December 2002
Extracted year: 2002
Found URL link: https://www.equinor.com/en/news/archive/2002/12/23/KristinContractsPlaced
Added URL link: https://www.equinor.com/en/news/archive/2002/12/23/KristinContractsPlaced under year 2002
Found date span: 23 December 2002
Extracted year: 2002
Found URL link: https://www.equinor.com/en/news/archive/2002/12/23/SigynOnStream
Added URL link: https://www.equinor.com/en/news/archive/2002/12/23/SigynOnStream under year 2002
Found date span: 20 December 2002
Extracted year: 2002
Found URL link: https://www.equinor.com/en/news/archive/2002/12/20/CleanupSkillsForSpain
Added URL link: https://www.equinor.com/en/news/archive/2002/12/20/CleanupSkillsForSpain under year 2002
Found date span:

In [77]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_html(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # List of content div classes that should be processed
        content_div_classes = [
            'articleBody', 
            'text-pretty', 
            'prose',
            'text parbase section',
            'ingress text parbase section'
        ]

        content = ''
        
        # Extract text from the main title if available
        main_title = soup.find('h1')
        if main_title:
            content += main_title.get_text(separator=' ', strip=True) + ' '

        for class_name in content_div_classes:
            content_divs = soup.find_all('div', class_=class_name)
            if content_divs:
                for div in content_divs:
                    paragraphs = div.find_all(['p', 'ul', 'li'])
                    if paragraphs:
                        content += ' '.join(p.get_text(separator=' ', strip=True) for p in paragraphs) + ' '

        # Clean up the content by stripping excess whitespace
        content = ' '.join(content.split())

        return content if content.strip() else None
    except Exception as e:
        print(f"Error processing HTML URL {url}: {e}")
        return None

# List to store extracted text for the year 2002
extracted_2002 = []

for link in links_2002:
    print(f"Processing URL: {link}")
    html_text = extract_text_from_html(link)
    if html_text:  # Only append if text was successfully extracted
        extracted_2002.append(html_text)

print(f"Total articles extracted for 2002: {len(extracted_2002)}")


Processing URL: https://www.equinor.com/en/news/archive/2002/12/31/OperatorshipsOnTheMove
Processing URL: https://www.equinor.com/en/news/archive/2002/12/23/KristinContractsPlaced
Processing URL: https://www.equinor.com/en/news/archive/2002/12/23/SigynOnStream
Processing URL: https://www.equinor.com/en/news/archive/2002/12/20/CleanupSkillsForSpain
Processing URL: https://www.equinor.com/en/news/archive/2002/12/20/NewTransportCompanyCreated
Processing URL: https://www.equinor.com/en/news/archive/2002/12/20/KvitebjornTopSideReady
Processing URL: https://www.equinor.com/en/news/archive/2002/12/20/HeidrunDrillingContractExtended
Processing URL: https://www.equinor.com/en/news/archive/2002/12/20/HistoricGassledAgreementSigned
Processing URL: https://www.equinor.com/en/news/archive/2002/12/20/GreenLightForHydroDevelopmentOnTampen
Processing URL: https://www.equinor.com/en/news/archive/2002/12/19/OrmenLangegasToGoViaSleipner
Processing URL: https://www.equinor.com/en/news/archive/2002/12/18/O

In [78]:
extracted_2002

['Operatorships on the move Operator responsibility for Snorre, Visund, Tordis and Vigdis in the Tampen area of the North Sea passed from Norsk Hydro to Statoil at midnight on 31 December. The two companies have been preparing for the change-over since the summer of 2000, reports Svein G Larsen, who has headed the transfer project in Exploration & Production Norway. “Work has gone very smoothly, with companies and union officials cooperating extremely well,” he says. “Everyone has put in a lot of effort to complete the process.” The transfer means that Statoil will become the only operator in Tampen area which also includes Statfjord and Gullfaks as well as a number of subsea developments tied back to the main fields. A million barrels of oil equivalent are delivered every day from this area, corresponding to 28 per cent of total Norwegian offshore output. The hand-over also brings Statoil 550 new and motivated employees, including about 250 employed offshore on Snorre A and B as well 

In [79]:
news_by_year["2002"] = extracted_2002

In [85]:
# Initialize WebDriver (e.g., for Chrome)
driver = webdriver.Chrome()

# Variable to store links for the year 2003
links_2003 = []

# URL for Equinor news archive for the year 2003
equinor_news_url = 'https://www.equinor.com/news?years=2003'

# Start the WebDriver and load the URL
driver.get(equinor_news_url)

while True:
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # Extract date headers and their respective links
    date_spans = soup.find_all('span', class_='sc-aac650bd-0 dakTuR')
    news_items = soup.find_all('li', class_='sc-db713ca2-0 ezdnkb sc-41ce2e98-1 dYECJz ais-Hits-item')

    for date_span, news_item in zip(date_spans, news_items):
        date_text = date_span.get_text().strip()
        print(f"Found date span: {date_text}")  # Debugging line to check the date text

        # Check if date_text contains a valid year
        if len(date_text.split()) > 1:
            year = date_text.split()[-1]  # Extract the year from the date text
            print(f"Extracted year: {year}")  # Debugging line to check the extracted year

            # Only process links for the year 2002
            if year == "2003":
                # Extracting the 'a' tags within the news item
                for a_tag in news_item.find_all('a', href=True):
                    href = a_tag['href']
                    full_url = f"https://www.equinor.com{href}" if href.startswith('/') else href
                    print(f"Found URL link: {full_url}")  # Debugging line to check found links

                    if full_url not in links_2003:
                        links_2003.append(full_url)
                        print(f"Added URL link: {full_url} under year 2003")  # Debugging line to check added links
        else:
            print(f"Invalid date format: {date_text}")
    
    # Check if the next page button is available and click it
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Next"]'))
        )
        # Check if the next button is disabled
        if 'disabled' in next_button.get_attribute('class'):
            print("Next button is disabled. No more pages to load.")
            break
        # Scroll the next button into view
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        # Use ActionChains to click the button
        ActionChains(driver).move_to_element(next_button).click(next_button).perform()
        time.sleep(2)  # Wait for content to load
    except Exception as e:
        print(f"No more 'Next' button found or error occurred: {e}")
        break  # No more "Next" button found or an error occurred

# Close the WebDriver
driver.quit()

# Print the collected links for the year 2003
print(f"Links for the year 2003: {links_2003}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found date span: 30 December 2003
Extracted year: 2003
Found URL link: https://www.equinor.com/en/news/archive/2003/12/30/ThroughtubingWellsCutCosts
Added URL link: https://www.equinor.com/en/news/archive/2003/12/30/ThroughtubingWellsCutCosts under year 2003
Found date span: 23 December 2003
Extracted year: 2003
Found URL link: https://www.equinor.com/en/news/archive/2003/12/23/AlgeriaAcquisitionApproved
Added URL link: https://www.equinor.com/en/news/archive/2003/12/23/AlgeriaAcquisitionApproved under year 2003
Found date span: 23 December 2003
Extracted year: 2003
Found URL link: https://www.equinor.com/en/news/archive/2003/12/23/PronovaBiocareToBeSold
Added URL link: https://www.equinor.com/en/news/archive/2003/12/23/PronovaBiocareToBeSold under year 2003
Found date span: 19 December 2003
Extracted year: 2003
Found URL link: https://www.equinor.com/en/news/archive/2003/12/19/OrmenLangeRockInstallationContractAwarded
Added URL link: https://www.equinor.com/en/news/archive/2003/12/19/

In [86]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_html(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # List of content div classes that should be processed
        content_div_classes = [
            'articleBody', 
            'text-pretty', 
            'prose',
            'text parbase section',
            'ingress text parbase section'
        ]

        content = ''
        
        # Extract text from the main title if available
        main_title = soup.find('h1')
        if main_title:
            content += main_title.get_text(separator=' ', strip=True) + ' '

        for class_name in content_div_classes:
            content_divs = soup.find_all('div', class_=class_name)
            if content_divs:
                for div in content_divs:
                    paragraphs = div.find_all(['p', 'ul', 'li'])
                    if paragraphs:
                        content += ' '.join(p.get_text(separator=' ', strip=True) for p in paragraphs) + ' '

        # Clean up the content by stripping excess whitespace
        content = ' '.join(content.split())

        return content if content.strip() else None
    except Exception as e:
        print(f"Error processing HTML URL {url}: {e}")
        return None

# List to store extracted text for the year 2002
extracted_2003 = []

for link in links_2003:
    print(f"Processing URL: {link}")
    html_text = extract_text_from_html(link)
    if html_text:  # Only append if text was successfully extracted
        extracted_2003.append(html_text)

print(f"Total articles extracted for 2003: {len(extracted_2003)}")


Processing URL: https://www.equinor.com/en/news/archive/2003/12/30/ThroughtubingWellsCutCosts
Processing URL: https://www.equinor.com/en/news/archive/2003/12/23/AlgeriaAcquisitionApproved
Processing URL: https://www.equinor.com/en/news/archive/2003/12/23/PronovaBiocareToBeSold
Processing URL: https://www.equinor.com/en/news/archive/2003/12/19/OrmenLangeRockInstallationContractAwarded
Processing URL: https://www.equinor.com/en/news/archive/2003/12/19/AViableRelationshipWithAmnesty
Processing URL: https://www.equinor.com/en/news/archive/2003/12/19/LicenceCarveoutInNorwegianSea
Processing URL: https://www.equinor.com/en/news/archive/2003/12/17/CleanWaterFromVigdis
Processing URL: https://www.equinor.com/en/news/archive/2003/12/17/SaleOfScanraffpartConcluded
Processing URL: https://www.equinor.com/en/news/archive/2003/12/16/CloserIntegrationWithNewCentre
Processing URL: https://www.equinor.com/en/news/archive/2003/12/16/RegrettableThatNordlandVIWillNotBeMadeAvailable
Processing URL: https:

In [91]:
for year in news_by_year:
    print(f"{year}: Number of articles = {len(news_by_year[year])}") ## den fikk altså ikke hentet ut absolutt alt, men mye!

2024: Number of articles = 79
2023: Number of articles = 123
2022: Number of articles = 191
2021: Number of articles = 187
2020: Number of articles = 157
2019: Number of articles = 157
2018: Number of articles = 171
2017: Number of articles = 153
2016: Number of articles = 173
2015: Number of articles = 158
2014: Number of articles = 140
2013: Number of articles = 160
2012: Number of articles = 168
2011: Number of articles = 189
2010: Number of articles = 156
2009: Number of articles = 172
2008: Number of articles = 117
2007: Number of articles = 291
2006: Number of articles = 406
2005: Number of articles = 375
2004: Number of articles = 362
2001: Number of articles = 320
2000: Number of articles = 255
1999: Number of articles = 329
1998: Number of articles = 154
2002: Number of articles = 342
2003: Number of articles = 323


In [90]:
news_by_year["2003"] = extracted_2003

In [92]:
news_by_year2 = dict(sorted(news_by_year.items(), key=lambda item: int(item[0]), reverse=False))

for year in news_by_year2:
    print(f"{year}: Number of articles = {len(news_by_year2[year])}") ## den fikk altså ikke hentet ut absolutt alt, men mye!


1998: Number of articles = 154
1999: Number of articles = 329
2000: Number of articles = 255
2001: Number of articles = 320
2002: Number of articles = 342
2003: Number of articles = 323
2004: Number of articles = 362
2005: Number of articles = 375
2006: Number of articles = 406
2007: Number of articles = 291
2008: Number of articles = 117
2009: Number of articles = 172
2010: Number of articles = 156
2011: Number of articles = 189
2012: Number of articles = 168
2013: Number of articles = 160
2014: Number of articles = 140
2015: Number of articles = 158
2016: Number of articles = 173
2017: Number of articles = 153
2018: Number of articles = 171
2019: Number of articles = 157
2020: Number of articles = 157
2021: Number of articles = 187
2022: Number of articles = 191
2023: Number of articles = 123
2024: Number of articles = 79


## Splitting up 

In [93]:
def split_up_text(article, n): #husk at n må være mindre enn 512 (helst 500) for at modellen skal bruke hele!
    words = article.strip().split()
    if len(words) <= n :
        return [article]
    else:
        new_article = []
        for i in range(0, len(words), n):
            chunk = words[i:i+n]
            new_article.append(' '.join(chunk))
        return new_article

In [94]:
def split_articles_content_by_year(news_by_year, n):
    split_articles_by_year = {}  # Define inside the function

    for year, articles in news_by_year.items():
        split_articles_by_year[year] = []
        for article in articles:
            split_content = split_up_text(article, n)
            split_articles_by_year[year].extend(split_content)

    return split_articles_by_year  # Return the dictionary

split_content = split_articles_content_by_year(news_by_year,450)

data = split_content

In [95]:
with open("Equinor_split_news_content_by_year.pkl","wb") as file:
    pickle.dump(split_content,file)

## Embedding

In [68]:
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

def embed_text(t): # obs dette tar lang tid
    embeddings = model.encode(t, convert_to_tensor=True)
    return embeddings

/opt/anaconda3/envs/embed/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [69]:

print(data.keys())
num_years = len(list(data.keys()))
print(f"Dette vil ta {(120*num_years)/60} minutter")

import time

annual_embeddings = {}
all_embeddings = []
for year, articles in data.items():
    print(year)
    annual_embeddings[year] = []
    start_time = time.time()  # Record start time
    annual_embeddings[year] = []
    for article in articles:
        if time.time() - start_time > 120:  # Check if more than 5 minutes have passed (300 seconds)
            print(f"Skipping year {year} due to time constraint.")
            break
        embedding = embed_text(article)
        annual_embeddings[year].append(embedding)



dict_keys(['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003', '2001', '2000', '1999', '1998'])
Dette vil ta 52.0 minutter
2024
Skipping year 2024 due to time constraint.
2023
Skipping year 2023 due to time constraint.
2022
Skipping year 2022 due to time constraint.
2021
Skipping year 2021 due to time constraint.
2020
Skipping year 2020 due to time constraint.
2019
Skipping year 2019 due to time constraint.
2018
Skipping year 2018 due to time constraint.
2017
Skipping year 2017 due to time constraint.
2016
Skipping year 2016 due to time constraint.
2015
Skipping year 2015 due to time constraint.
2014
Skipping year 2014 due to time constraint.
2013
Skipping year 2013 due to time constraint.
2012
Skipping year 2012 due to time constraint.
2011
Skipping year 2011 due to time constraint.
2010
Skipping year 2010 due to time constraint.
2009
Skipping year 2009 due to ti

In [97]:
del annual_embeddings["2003"]

In [98]:
annual_embeddings.keys()

dict_keys(['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2001', '2000', '1999', '1998'])

In [ ]:
embedded_2002 = []
start_time = time.time()  # Record start time

for article in extracted_2002:
    if time.time() - start_time > 120:  # Check if more than 2 minutes have passed
        print(f"Skipping the rest of 2002 articles due to time constraint.")
        break
    embedding = embed_text(article)
    embedded_2002.append(embedding)

embedded_2002

In [101]:
annual_embeddings["2002"] = embedded_2002

In [102]:
embedded_2003 = []
start_time = time.time()  # Record start time

for article in extracted_2003:
    if time.time() - start_time > 120:  # Check if more than 2 minutes have passed
        print(f"Skipping the rest of 2003 articles due to time constraint.")
        break
    embedding = embed_text(article)
    embedded_2003.append(embedding)

Skipping the rest of 2002 articles due to time constraint.


In [103]:
annual_embeddings["2003"] = embedded_2003

In [104]:
annual_embeddings.keys()

dict_keys(['2024', '2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2001', '2000', '1999', '1998', '2002', '2003'])

In [106]:
annual_embeddings = dict(sorted(annual_embeddings.items(), key=lambda item: int(item[0]), reverse=False))

annual_embeddings.keys()

dict_keys(['1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'])

In [107]:
with open("equinor_news_embeddings.pkl","wb") as file:
    pickle.dump(annual_embeddings,file)

## Fikser lista med split content!

In [3]:
with open("data/Equinor_split_news_content_by_year.pkl","rb") as file:
    equinor_split_news = pickle.load(file)

In [ ]:
equinor_split_news = dict(sorted(equinor_split_news.items(), key=lambda item: int(item[0]), reverse=False))

for year in equinor_split_news:
    print(f"{year}: Number of articles = {len(equinor_split_news[year])}") ## den fikk altså ikke hentet ut absolutt alt, men mye!


In [5]:
with open("Equinor_split_news_content_by_year.pkl","wb") as file:
    pickle.dump(equinor_split_news,file)